In [211]:
import torch
from utils.config import Hyperparameters as hparams


In [212]:
x = torch.tensor([7, 8, 9])
y = torch.tensor([1, 2, 3])
z = torch.tensor([[9, 4, 2],[3, 0, 2],[4, 7, 5]])
z.shape

torch.Size([3, 3])

In [213]:
result = []
for i in range(z.shape[0]):
    v = y - x
    w = z[i] - x
    t = torch.clamp(torch.dot(v, w) / torch.dot(v, v), 0, 1)
    dist = torch.norm(x + t * v - z[i])
    result.append(dist)
result

[tensor(6.4807), tensor(3.), tensor(2.1602)]

In [214]:
v

tensor([-6, -6, -6])

In [215]:
v = y - x
w = z - x.unsqueeze(0)
t = torch.clamp(torch.sum(v * w, dim = -1) / torch.sum(v * v), 0, 1)
dist = torch.norm(x.unsqueeze(0) + t.unsqueeze(-1) * v - z, dim = -1)
dist

tensor([6.4807, 3.0000, 2.1602])

In [216]:
x = torch.tensor([7, 8, 9], dtype = torch.float32)
y = torch.tensor([[1, 2, 3], [2, 3, 1], [4, 9, 3]], dtype = torch.float32)
z = torch.tensor([[9, 4, 2], [3, 0, 2], [4, 7, 5]], dtype = torch.float32)

In [217]:
ls = []
for i in range(y.shape[0]):
    v = y[i] - x
    w = z - x
    t = torch.clamp(torch.sum(v * w, dim = -1) / torch.sum(v * v), 0, 1)
    dist = torch.norm(x.unsqueeze(0) + t.unsqueeze(-1) * v - z, dim = -1)
    ls.append(dist)
ls

[tensor([6.4807, 3.0000, 2.1602]),
 tensor([5.5488, 3.3166, 1.5102]),
 tensor([6.8366, 9.1104, 1.9337])]

In [218]:
v = y - x
w = z - x
t = torch.clamp(torch.inner(v, w) / torch.sum(v * v, dim = -1).unsqueeze(1), 0, 1)
dist = torch.norm(x.unsqueeze(0).unsqueeze(1) + t.unsqueeze(-1) * v.unsqueeze(1) - z, dim = -1)
dist


tensor([[6.4807, 3.0000, 2.1602],
        [5.5488, 3.3166, 1.5102],
        [6.8366, 9.1104, 1.9337]])

In [223]:
bk_val = torch.tanh_(0.2*torch.min(dist, dim=1).values)
xy_dist = torch.norm(x - y, dim=1)
chan_gain = bk_val * hparams.beta_1 / xy_dist+ (1 - bk_val) * hparams.beta_2 / (xy_dist ** 1.65)
chan_gain

tensor([9.1942e-07, 6.5702e-07, 1.2973e-06])

In [225]:
torch.mean(hparams.P_AVG * chan_gain / hparams.noise)

tensor(48009.4180)